# 股利法

In [154]:
#匯入股票資訊
import datetime
import pandas as pd
import warnings
import requests
from io import StringIO
import pandas_profiling

#抓取所有股票當下殖利率
def crawler(date):
    datestr = date.strftime('%Y%m%d')
    url = 'https://www.twse.com.tw/exchangeReport/BWIBBU_d?response=csv&date='+datestr+'&selectType=ALL'
    res = requests.get(url)
    df = pd.read_csv(StringIO(res.text), header=1)
    df['本益比'] = pd.to_numeric(df['本益比'], errors='coerce')
    
    return df.dropna(thresh=3).dropna(thresh=0.8, axis=1)

df = crawler(datetime.date(2021,6,11)) #注意要是開盤日
df.head()

,證券代號,證券名稱,殖利率(%),股利年度,本益比,股價淨值比,財報年/季
0,1101,台泥,6.81,109.0,12.30,1.48,110/1
1,1102,亞泥,7.00,109.0,9.92,1.13,110/1
2,1103,嘉泥,6.33,109.0,7.70,0.57,110/1
3,1104,環泥,4.85,109.0,10.37,0.79,110/1
4,1108,幸福,5.13,109.0,15.69,1.25,110/1


In [166]:
#判斷式
title=df['證券代號'][0:]
Yield=df['殖利率(%)'][0:]

cheep= [i for i in range(952) if Yield[i] >= 6.6]
reasonable= [i for i in range(952) if Yield[i] < 6.6 and Yield[i] > 3.3 ]
expensive = [i for i in range(952) if  Yield[i] <= 3.3 ]
print('便宜價的股票代號')
print(title[cheep])
print('合理價的股票代號')
print(title[reasonable])
print('昂貴價的股票代號')
print(title[expensive])
print('--------------------------------------------')

if any("2886" in item for item in title[cheep]): 
    has_string = True 
    print ("2886兆豐金 : 便宜價")   
if any("2886" in item for item in title[reasonable]): 
    has_string = True 
    print ("2886兆豐金 : 合理價")
if any("2886" in item for item in title[expensive]): 
    has_string = True 
    print ("2886兆豐金 : 昂貴價")


便宜價的股票代號
0      1101
1      1102
33     1309
38     1315
44     1325
       ... 
875    8112
882    8213
907    8481
927    9919
948    9945
Name: 證券代號, Length: 75, dtype: object
合理價的股票代號
2      1103
3      1104
4      1108
5      1109
9      1210
       ... 
943    9940
944    9941
945    9942
946    9943
949    9946
Name: 證券代號, Length: 344, dtype: object
昂貴價的股票代號
6      1110
7      1201
8      1203
10     1213
13     1217
       ... 
938    9934
941    9938
947    9944
950    9955
951    9958
Name: 證券代號, Length: 533, dtype: object
--------------------------------------------
2886兆豐金 : 合理價


# 高低價法

In [23]:
#股票價格
import pandas_datareader.data as pdr

data = pdr.DataReader(name = "2886.TW" ,data_source="yahoo",start="1/1/2019",end="6/18/2021")
print(data.head())
print(data.shape)

close = data["Close"][0:]
print(close)

                 High        Low       Open      Close      Volume  Adj Close
Date                                                                         
2019-01-02  25.700001  25.299999  25.700001  25.450001  12983080.0  21.548540
2019-01-03  25.500000  25.200001  25.299999  25.450001  12343811.0  21.548540
2019-01-04  25.400000  25.200001  25.250000  25.250000  12103342.0  21.379198
2019-01-07  25.600000  25.450001  25.549999  25.600000  14028735.0  21.675545
2019-01-08  25.650000  25.450001  25.650000  25.500000  13281695.0  21.590872
(593, 6)
Date
2019-01-02    25.450001
2019-01-03    25.450001
2019-01-04    25.250000
2019-01-07    25.600000
2019-01-08    25.500000
                ...    
2021-06-11    32.900002
2021-06-15    32.750000
2021-06-16    33.000000
2021-06-17    32.900002
2021-06-18    32.750000
Name: Close, Length: 593, dtype: float64


In [90]:
#判斷式
import statistics
mean = statistics.mean(close)
if close[592] > mean:
    print("2886兆豐金 : 昂貴價")
if close[592] < mean:  
    print("2886兆豐金 : 便宜價")
if close[592] == mean:  
    print("2886兆豐金 : 合理價")


2886兆豐金 : 昂貴價
29.93465425513848


# 本淨比法

In [146]:
#from goodinfo 爬蟲 PBR平均
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 

options = Options()
options.add_argument("--disable-notifications")
options.add_argument("headless")
chrome = webdriver.Chrome('chromedriver', chrome_options=options)
chrome.get("https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2886")

now_price = chrome.find_element_by_css_selector('#divDetail > table > tbody > tr:nth-child(3) > td:nth-child(4) > nobr > a')
worth = chrome.find_element_by_css_selector('#divDetail > table > tbody > tr:nth-child(3) > td:nth-child(21) > nobr > a')
PBR_high = chrome.find_element_by_css_selector('#txtFinAvgData > div > div > table > tbody > tr:nth-child(7) > td:nth-child(15)')
PBR_avg = chrome.find_element_by_css_selector('#txtFinAvgData > div > div > table > tbody > tr:nth-child(7) > td:nth-child(13)')
PBR_low = chrome.find_element_by_css_selector('#txtFinAvgData > div > div > table > tbody > tr:nth-child(7) > td:nth-child(14)')

# 將取到的資料印出來
print('目前價格:'+now_price.text +'\t目前淨值:'+ worth.text+ '\t最高PBR:' + PBR_high.text+ '\t平均PBR:' + PBR_avg.text + '\t最低PBR:' + PBR_low.text)

C:\Users\Coco_wu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


目前價格:32.75	目前淨值:24.29	最高PBR:1.97	平均PBR:1.55	最低PBR:1.21


In [144]:
#判斷式
price_now = float(now_price.text)
expensive_PBR = float(PBR_high.text) * float(worth.text)
cheep_PBR = float(PBR_low.text) * float(worth.text)
if price_now >= expensive_PBR:
    print("2886兆豐金 : 昂貴價")
if price_now <= cheep_PBR:  
    print("2886兆豐金 : 便宜價")
if price_now > cheep_PBR and price_now < expensive_PBR:  
    print("2886兆豐金 : 合理價")
chrome.quit()

2886兆豐金 : 合理價


# 本益比法

In [147]:
#from goodinfo 爬蟲 PBR平均
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 

options = Options()
options.add_argument("--disable-notifications")
options.add_argument("headless")
chrome = webdriver.Chrome('chromedriver', chrome_options=options)
chrome.get("https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2886")

now_price = chrome.find_element_by_css_selector('#divDetail > table > tbody > tr:nth-child(3) > td:nth-child(4) > nobr > a')
now_eps = chrome.find_element_by_css_selector('#divDetail > table > tbody > tr:nth-child(3) > td:nth-child(19) > nobr > a')
Annual_increase_eps = chrome.find_element_by_css_selector('#divDetail > table > tbody > tr:nth-child(3) > td:nth-child(20) > nobr > a')
PER_high = chrome.find_element_by_css_selector('#txtFinAvgData > div > div > table > tbody > tr:nth-child(7) > td:nth-child(10)')
PER_avg = chrome.find_element_by_css_selector('#txtFinAvgData > div > div > table > tbody > tr:nth-child(7) > td:nth-child(8)')
PER_low = chrome.find_element_by_css_selector('#txtFinAvgData > div > div > table > tbody > tr:nth-child(7) > td:nth-child(9)')

# 將取到的資料印出來
print('目前價格:'+now_price.text +'\t目前eps:'+ now_eps.text+'\t年增eps:'+ Annual_increase_eps.text+ '\t最高PER:' + PER_high.text+ '\t平均PER:' + PER_avg.text + '\t最低PER:' + PER_low.text)

C:\Users\Coco_wu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


目前價格:32.75	目前eps:0.46	年增eps:+0.32	最高PER:21.78	平均PER:17.16	最低PER:13.34


In [151]:
#判斷式
price_now = float(now_price.text)
fultrue_eps= float(now_eps.text)+float(Annual_increase_eps.text)
expensive_PER = float(PER_high.text) * float(fultrue_eps)
cheep_PER = float(PER_low.text) * float(fultrue_eps)
if price_now >= expensive_PER:
    print("2886兆豐金 : 昂貴價")
if price_now <= cheep_PER:  
    print("2886兆豐金 : 便宜價")
if price_now > cheep_PER and price_now < expensive_PER:  
    print("2886兆豐金 : 合理價")
chrome.quit()

2886兆豐金 : 昂貴價


In [171]:
#使用selenium套件爬蟲
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

options = Options()
options.add_argument("--disable-notifications")
options.add_argument("headless")
chrome = webdriver.Chrome('chromedriver', chrome_options=options)
chrome.get("https://goodinfo.tw/StockInfo/StockBzPerformance.asp?STOCK_ID=2886")

#填充表單_PER/PBR
s1 = Select(chrome.find_element_by_css_selector('body > table:nth-child(8) > tbody > tr > td:nth-child(3) > table.b1.r10_0 > tbody > tr > td > table > tbody > tr > td:nth-child(1) > nobr:nth-child(1) > select'))
s1.select_by_index(2) 
#等待運行時間
import time
time.sleep(2)

from bs4 import BeautifulSoup
soup = BeautifulSoup(chrome.page_source, 'html.parser')
data = soup.select_one("#txtFinDetailData")

import pandas as pd
dfs = pd.read_html(data.prettify())
df = dfs[0]
df


C:\Users\Coco_wu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,年度,股本 (億),財報 評分,年度股價(元),年度股價(元),年度股價(元),年度股價(元),年度股價(元),年度股價(元),本益比(PER)統計,本益比(PER)統計,本益比(PER)統計,本益比(PER)統計,本淨比(PBR)統計,本淨比(PBR)統計,本淨比(PBR)統計,本淨比(PBR)統計
1,年度,股本 (億),財報 評分,最高,最低,收盤,平均,漲跌,漲跌 (%),EPS (元),最高 PER,最低 PER,平均 PER,BPS (元),最高 PBR,最低 PBR,平均 PBR
2,21Q1,1360,50,33.2,28.15,32.6,31.2,+2.8,+9.4,0.46,18,15.3,16.9,24.29,1.37,1.16,1.28
3,2020,1360,47,33.8,26.2,29.8,30.1,-0.8,-2.6,1.84,18.4,14.2,16.3,24.01,1.41,1.09,1.25
4,2019,1360,50,32.5,25.2,30.6,29.3,+4.65,+17.9,2.13,15.3,11.8,13.7,23.83,1.36,1.06,1.23
5,2018,1360,47,27.8,23.7,25.95,26,+1.9,+7.9,2.07,13.4,11.4,12.6,23.11,1.2,1.03,1.13
6,2017,1360,50,26.4,22.8,24.05,24.2,+1.05,+4.6,1.89,14,12.1,12.8,21.92,1.2,1.04,1.1
7,2016,1360,50,26.2,19.05,23,22.8,+1.75,+8.2,1.65,15.9,11.5,13.8,21.47,1.22,0.89,1.06
8,2015,1360,53,28.5,20.9,21.25,24.8,-3.2,-13.1,2.35,12.1,8.89,10.6,21.57,1.32,0.97,1.15
9,2014,1245,56,27,22.6,24.45,24.6,-0.65,-2.6,2.43,11.1,9.29,10.1,20.97,1.29,1.08,1.17
